In [ ]:
from buttermilk import BM
from promptflow.tracing import trace, start_trace
from rich import print as rprint

import os

from hydra import initialize, compose
from omegaconf import OmegaConf

# Load config, specifying overrides for our particular job
with initialize(version_base=None, config_path="./conf"):
    cfg = compose(config_name='config',
                  overrides=["+data=drag", "+step=ordinary"])

# Load the main ButterMilk singleton instance
# This takes care of credentials, save paths, and other defaults
bm = BM(cfg=cfg)

# Print config
rprint(OmegaConf.to_container(bm.cfg, resolve=True))



2024-11-04 13:05:45 J5HW6L4KT6 buttermilk buttermilk.py[ 182] DEBUG Config passed in but we already have one loaded. Overwriting.


{
    'name': 'automod',
    'job': 'judge',
    'secret_provider': 'azure',
    'logger': 'gcp',
    'verbose': True,
    'models_secret': 'models',
    'save_dest': 'gcp',
    'save_dir': None,
    'gcp': {'project': 'dmrc-analysis', 'region': 'us-central1', 'bucket': 'dmrc-analysis'},
    'azure': {'vault': 'https://suzorvault.vault.azure.net/', 'resource_group': 'rg-suzor_ai'},
    'aws': {'secret_name': 'secret', 'region': 'us-east-1'},
    'data': {
        'drag': {
            'name': 'drag queens - alt text',
            'type': 'file',
            'uri': 'gs://dmrc-platforms/data/drag_train.jsonl',
            'columns': {'record_id': 'id', 'content': 'alt_text', 'groundtruth': 'expected'}
        }
    },
    'step': [
        {
            'judge': {
                'num_runs': 1,
                'concurrent': 20,
                'init': {
                    'template': 'judge',
                    'criteria': 'criteria_ordinary',
                    'formatting': 'json_rules',
                    'model': ['haiku']
                }
            }
        }
    ],
    'run': {'platform': 'local'}
}

In [ ]:
# Create an orchestrator to conduct all combinations of jobs we want to run
from buttermilk.runner.orchestrator import MultiFlowOrchestrator

orchestrator = MultiFlowOrchestrator()


In [13]:
os.getcwd()


'/Users/suzor/src/buttermilk/examples/automod'